# Scaling Analysis under oscillating oxygen environments

Using this scaling analysis script allows for applying your single analysis script to a large amount of image sequences organized in the OMERO `project` or `dataset` structures. Therefore, your custom developed analyses can scale to large image volumes without you touching or changing the code. 

## 1. Setup

Define the `omero_id` and `omero_type` of the image data you would like to process. The `omerod_id` is the number you can find in the top right corner when selecting a OMERO `project`, `dataset` or `image` in the `OMERO Web` application. The `omero_type` must be `project` or `dataset` when the OMERO id points to a project or dataset and `image` if it is just a single image! Please note that if you define the wrong `omero_type` you will get an error lateron 🤯!

Also provide your credentials for the OMERO server!

In [ ]:
import os

# OMERO resource that you want to analyze
omero_type = "dataset" # can be "image", "project" or "dataset"
omero_id = 3446 # change the id if you want to apply the analysis to a different omero resource

# your omero credentials
username = "<your username>"
password = "<your password>"

# do not change the lines below
assert username != "<your username>", "Please replace '<your username>' with your OMERO username"
assert password != "<your password>", "Please replace '<your password>' with your OMERO username"

import logging

if not "OMERO_SERVER" in os.environ:
    logging.warning("No 'OMERO_SERVER' defined. Fallback to default OMERO_SERVER address 'omero'! This can lead to connection faults!")
if not "OMERO_WEB" in os.environ:
    logging.warning("No 'OMERO_WEB' defined. Links to view OMERO data in web viewer might not work!")

credentials = dict(
    serverUrl=os.environ.get('OMERO_SERVER', 'omero'),
    username=username,
    password = password
)

omero_cred = dict(
    host = credentials['serverUrl'],
    username = credentials['username'],
    passwd = credentials['password']
)

## 1.2 Specify the analysis script

Now you have to specify the name of the analysis script you want to apply to the image data. At best copy the script to the same location as this script! Then you only have to specify the name of the script!

**Note:** If the analysis script is not located in the same folder you need to specify the path to it.

In [ ]:
analysis_script = "GrowthRate_oscillation_oxygen.ipynb"

# 2. Information about the underlying data

We summarize the amount of underlying data

In [ ]:
from acia.segm.omero.utils import list_image_ids_in
from omero.gateway import BlitzGateway

with BlitzGateway(**omero_cred) as conn:
    image_ids = list_image_ids_in(omero_id, omero_type, conn)

## TODO: give an overview about the data
print(image_ids)

# 3. Scale the analysis script to all image sequences

Now we apply the analysis script to every image sequence individually 🚀! You can lean back and enjoy the working computer 😎 🥂

**Note:** For heavy analysis scripts or for larget `datasets` or `projects` this process may take a while (from minutes to hours or days). The top-level progress bar will indicate the total progress and give you an indication how long this will take. For large image data volumes we can recommend execution over night 🌔!

In [ ]:
from datetime import datetime
from pathlib import Path
from acia.analysis import scale

# set the base path for all results
stem = Path(analysis_script).stem
output_path = Path("./automated_executions") / stem / datetime.today().isoformat()

print(f"Results are stored in: {output_path.absolute()}")

# scale your analysis script to many images
result = scale(output_path, analysis_script=analysis_script, image_ids=image_ids)

# 4. Inspect your analysis results


In [ ]:
import urllib.parse
from IPython.display import Video, Markdown, display

base_url = os.environ.get("JUPYTERHUB_SERVICE_PREFIX", None)

if base_url is None:
    url = f"file://{output_path.absolute()}"
else:
    url = f"{base_url}lab/tree/{urllib.parse.quote(str(output_path))}"

output = f"""# Inspect your analyses
You can find all the individual analysis scripts here: <a href="{url}">{url}</a>"""

display(Markdown(output))

# 5. Generate Summary Statistics

In this section you can generate your custom summary statistics that combine the results of all experiment analyses. Just design the analysis script that you scaled above such that it outputs the results into a local files. Here, these results can be loaded, merged together and further processed or visualized!

First, "result_growth-rate.csv" files are collected from all the analyzed chambers, then mean and std of growth rate are calculated

In [ ]:
from pathlib import Path
import pandas as pd
import os, glob
import numpy as np

directory = Path("./automated_executions") / stem

# Find latest folder
latest_folder = Path(max(glob.glob(os.path.join(directory, '*/')), key=os.path.getmtime))
print(latest_folder)

In [ ]:
dfs = []

# if the csv. file exists, take growth rates, otherwise go to the next loop
for sub_folder in latest_folder.glob("execution*"):
    data_folder =  sub_folder / "tmp"
    data_files = os.listdir(data_folder)
    if "result_growth-rate.csv" not in data_files:
        print(sub_folder.name, 'was not analyzed')
    else:
        sub_df = pd.read_csv(data_folder / "result_growth-rate.csv", delimiter = ';')
        sub_df.loc[len(sub_df)] = {'m': sub_folder.name, 'b': sub_folder.name} # adding a new row (ImageID) to sub_df
        dfs.append(sub_df[['m']].T)

joint_df = pd.concat(dfs, ignore_index=True)
joint_df.columns = ['Cell number', 'Cell area', 'ImageID']
print(joint_df)

# calculate mean and std of growth rate
mean = [np.mean(joint_df['Cell number']), np.mean(joint_df['Cell area'])]
std = [np.std(joint_df['Cell number']), np.std(joint_df['Cell area'])]

statistics_df = pd.DataFrame({'': ['mean', 'std'],
                              'Cell number': [mean[0], std[0]],
                              'Cell area': [mean[1], std[1]]})

joint_df.to_csv('./ growth-rate_summary.csv', decimal='.', sep=';')
statistics_df.to_csv('./ growth-rate_mean-std.csv', decimal='.', sep=';')
print(statistics_df)

In [ ]:
# Growth rate summary in the box plot

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

fig, ax = plt.subplots(figsize=(5,5))
sns.boxplot(data=joint_df)
ax.set_ylabel('Growth rate [h$^{-1}$]')
ax.grid()

Next, "result.csv" files are collected from all the analyzed chambers, then growth is characterized by calculating instantaneous growth rate ($µ_{\Delta t}$), etc.

In [ ]:
# Change parameters here
dt_image = 1/360   # image acquisition interval (hour)
dt_switch = 30/60   # oxygen switch interval (hour)
t_start = 1   # analysis start (hour)
t_end = 3   # analysis end (hour)

n_switch = round((t_end - t_start) / dt_switch)   # number of switching

In [ ]:
from pathlib import Path
import pandas as pd
import os, glob
import numpy as np
import matplotlib.pyplot as plt

# if the result.csv file exists, take data and calculate µ_t
growth_df = pd.DataFrame()
growth_t_df = pd.DataFrame()
growth_t_df_MA = pd.DataFrame()
data_list_t = []
pop_area_df = pd.DataFrame()
cell_area_mean_df = pd.DataFrame()
cell_area_std_df = pd.DataFrame()
cell_number_df = pd.DataFrame()

for sub_folder in latest_folder.glob("execution*"):
    data_list = []
    data_folder =  sub_folder / "tmp"
    data_files = os.listdir(data_folder)
    
    if "result.csv" not in data_files:
        print(sub_folder.name, 'was not analyzed')
        
    else:
        sub_df = pd.read_csv(data_folder / "result.csv", delimiter = ';')
        
        # data smoothing by centered moving average (MA) for µ_t calculation
        sub_df['moving_average'] = sub_df['area_sum'].rolling(window=5, center=True).mean()
        sub_df['moving_average_single'] = sub_df['area_mean'].rolling(window=5, center=True).mean()
        
        # calculate µ_t (transient µ, first derivative of growth curve)
        µt_df = sub_df[(sub_df['time'] >= t_start) & (sub_df['time'] <= t_end)]
        y_area = np.log(µt_df['area_sum'])
        dy_area = np.gradient(y_area, dt_image)
        dy_area_series = pd.Series(dy_area)
        growth_t_df[sub_folder.name] = dy_area_series
        
        # calculate µ_t from moving-averaged area sum
        y_area_MA = np.log(µt_df['moving_average'])
        dy_area_MA = np.gradient(y_area_MA, dt_image)
        dy_area_series_MA = pd.Series(dy_area_MA)
        growth_t_df_MA[sub_folder.name] = dy_area_series_MA
        
        # calculate µ_step (µ for each switching step)
        for i in range(n_switch):
            min_time_i = t_start + i * dt_switch
            max_time_i = min_time_i + dt_switch
            timed_df_i = sub_df[(sub_df['time'] > min_time_i) & (sub_df['time'] < max_time_i)]
            m, b = np.polyfit(timed_df_i['time'], np.log(timed_df_i['area_sum']), 1)
            data_list.append({sub_folder.name : m})
            
        data_list_df = pd.DataFrame(data_list)
        growth_df = pd.concat([growth_df, data_list_df], axis=1)
        
        # collect 'area_sum' & 'area_mean' & 'area_std' from all the analyzed chambers
        pop_area_df[sub_folder.name] = sub_df['area_sum']
        cell_area_mean_df[sub_folder.name] = sub_df['area_mean'] 
        cell_area_std_df[sub_folder.name] = sub_df['area_std']
        cell_number_df[sub_folder.name] = sub_df['counts']

        
# save summary of population area, then save
pop_area_df.insert(0, 'time', sub_df['time'])
pop_area_df['mean'] = pop_area_df.mean(axis=1)
pop_area_df['std'] = pop_area_df.std(axis=1)
pop_area_df.to_csv('./ population-area_summary.csv', decimal='.', sep=';')

# calculate mean & std of µ_t, then save
growth_t_df['mean'] = growth_t_df.mean(axis=1)
growth_t_df['std'] = growth_t_df.std(axis=1)
time_series = np.arange(t_start, t_end + dt_image, dt_image)
growth_t_df.insert(0, 'time', time_series)
growth_t_df.to_csv('./ derivative-growth-rate_summary.csv', decimal='.', sep=';')

# calculate mean & std of µ_t (moving averaged), then save
growth_t_df_MA['mean'] = growth_t_df_MA.mean(axis=1)
growth_t_df_MA['std'] = growth_t_df_MA.std(axis=1)
growth_t_df_MA.insert(0, 'time', time_series)
growth_t_df_MA.to_csv('./ derivative-growth-rate_MA_summary.csv', decimal='.', sep=';')

# calculate mean & std of all aerobic & anaerobic growth
aerobic_rows = growth_df.iloc[0::2]   # even rows
anaerobic_rows = growth_df.iloc[1::2]   # odd rows

mean_aerobic = aerobic_rows.values.flatten().mean()
std_aerobic = aerobic_rows.values.flatten().std()
mean_anaerobic = anaerobic_rows.values.flatten().mean()
std_anaerobic = anaerobic_rows.values.flatten().std()

# save growth values
array = np.array([[mean_aerobic, std_aerobic], [mean_anaerobic, std_anaerobic]])
index_values = ['aerobic', 'anaerobic']
column_values = ['mean', 'std']
all_growth_df = pd.DataFrame(data = array,
                              index = index_values,
                              columns = column_values)

print(all_growth_df)
all_growth_df.to_csv('./ step-growth-rate_mean-std.csv', decimal='.', sep=';')

# calculate mean & std of aerobic & anaerobic growth for each step
growth_df['mean'] = growth_df.mean(axis=1)
growth_df['std'] = growth_df.std(axis=1)
growth_df.to_csv('./ step-growth-rate_summary.csv', decimal='.', sep=';')

In [ ]:
# Instantaneous growth rate summary in a plot

import matplotlib.ticker as ticker

# Set values for plotting
x = growth_t_df_MA['time']
y = growth_t_df_MA['mean']
yerr = growth_t_df_MA['std']

# Make plot
fig, axs = plt.subplots(figsize=(10, 5))
tick_spacing = dt_switch

fig.suptitle('µ_t')
axs.plot(x, y, color='gray', label='mean')
axs.fill_between(x, y - yerr, y + yerr, alpha=0.2, color='gray', label='Std.')
axs.set_xlabel('Time [h]')
axs.set_ylabel('µ_t [1/h]')
axs.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
axs.grid(axis = 'x')
axs.legend(loc='upper right')
plt.savefig('./ instantaneous-growth-rate.png')

In [ ]:
# Segment data into periods for polar plot

df_original = pd.read_csv('_derivative-growth-rate_MA_summary.csv')

dt_switch_min = 30   # switching duration in minute (half period)
t_polar = 6 * 2 * dt_switch_min   # number of row in a period (imaging every 10 seconds)
t_half = 6* dt_switch_min   # number of row in half a period (imaging every 10 seconds)

df_polar = pd.DataFrame()
df_aerobic = pd.DataFrame()
df_anaerobic = pd.DataFrame()

# Segment the DataFrame into chunks and assign to new columns
for i in range(0, len(df_original)-t_polar, t_polar):
    
    # Slice the DataFrame: rows from i to i+t_polar
    segment = df_original['mean'].iloc[i:i+t_polar+1].reset_index(drop=True)
    
    # Add the segment as a new column in the new DataFrame
    df_polar[f'period_{i//t_polar}'] = segment

df_polar['mean'] = df_polar.mean(axis=1)
df_polar['std'] = df_polar.std(axis=1)

df_polar.to_csv('./ derivative-growth-rate_MA_polar.csv', decimal='.', sep=';')